In [ ]:
from imageanalysis.making_stacks import *
dir1 = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/'
dir2 = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/'
# process_directory_h5(dir1, output_file='different_sides.h5',sort_by='time')
process_directory_h5(dir2, output_file='same_sides.h5',sort_by='time')

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 
 
different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'
same_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/same_sides.h5'

from imageanalysis.image_processing import CalibratedImages
calibrated_images = CalibratedImages(same_sides_irradiation, font_path=font_path)

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

for_debugging_area = '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-23_After_LaserCleaning_PtEvaporation/stack.h5'
from imageanalysis.feature_analysis import FeaturesAnalysis
features_analysis = FeaturesAnalysis(for_debugging_area, font_path=font_path)

In [ ]:
%matplotlib qt5
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'

same_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/same_sides.h5'

from imageanalysis.feature_analysis import FeaturesAnalysis

features_analysis = FeaturesAnalysis(different_sides_irradiation, font_path=font_path)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

datafile = 'Sample_2476/cluster_density_analysis/2l_gr.csv'
# datafile = 'Sample_2525/cluster_density_analysis/4l_gr.csv'
# datafile = 'analysis_results.csv'
datafile = 'Sample_2476/new_analysis/cda/kmeans_method/2L_gr.csv'
df = pd.read_csv(datafile)
# df['Roundness']
# df.head()
df

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import ast
from itertools import chain
import numpy as np
from scipy.signal import savgol_filter
from scipy.stats import norm, beta, gamma, lognorm, gaussian_kde
def get_shape_metric_info(sh_metric):
    """Converts string-encoded list column into flat numeric list."""
    name = sh_metric.name
    shape_metric = sh_metric.apply(ast.literal_eval)
    values = list(chain.from_iterable(shape_metric))
    return values, name

def plot_shape_metric_distribution(sh_metric, name, n_layers=None, as_line=False):
    """Plot single shape metric as histogram or line curve."""
    plt.figure(figsize=(10, 6))
    
    if as_line:
        counts, bins = np.histogram(sh_metric, bins=50)
        centers = (bins[:-1] + bins[1:]) / 2
        plt.plot(centers, counts, lw=2, color='blue')
    else:
        plt.hist(sh_metric, bins=50, density=True, alpha=0.8, color='blue')

    title = f'{name.replace("_", " ").title()} Distribution'
    if n_layers:
        title += f' ({n_layers} Layers)'
    plt.title(title)
    plt.xlabel(name.replace("_", " ").title())
    plt.ylabel('Density')
    plt.grid()
    plt.show()


def plot_shape_metric_comparison(metric_list, n_layers=None, as_line=False, smooth=True, window_length=15, polyorder=5):
    """Plot histogram or smoothed line for multiple shape metrics."""
    fig, ax = plt.subplots(1, len(metric_list), figsize=(5 * len(metric_list), 5))
    if len(metric_list) == 1:
        ax = [ax]

    for i, (sh_metric, name) in enumerate(metric_list):
        counts, bins = np.histogram(sh_metric, bins=100, density=False)
        centers = (bins[:-1] + bins[1:]) / 2
        kde = gaussian_kde(sh_metric)
        pdf = kde(centers)
        factor = np.sum(counts * pdf) / np.sum(pdf ** 2)
        pdf *= factor  # make the PDF to match histogram area
        #  mean and std for Gaussian fit
        dx = centers[1] - centers[0]
        pdf_norm  = pdf / np.sum(pdf) / dx  # normalize to area=1
        mean_pdf = np.sum(centers * pdf_norm) * dx
        std_pdf = np.sqrt(np.sum((centers - mean_pdf) ** 2 * pdf_norm) * dx)
        if as_line:
            if smooth:
                # Ensure window length is valid
                wl = min(window_length, len(centers) - 1 if len(centers) % 2 == 0 else len(centers))
                if wl % 2 == 0:
                    wl -= 1
                if wl >= 5:
                    counts_smooth = savgol_filter(counts, window_length=wl, polyorder=polyorder)
                else:
                    counts_smooth = counts  # fallback if too short
                ax[i].plot(centers, counts_smooth, lw=2, color='blue', label='Smoothed')
            else:
                ax[i].plot(centers, counts, lw=2, color='blue', label='Raw Line')
        else:
            ax[i].hist(sh_metric, bins=100, alpha=0.8, color='blue', label='Histogram')

        # mu, std = norm.fit(sh_metric)
        # gaussian = norm.pdf(centers, mu, std)
        # factor = np.sum(counts * gaussian) / np.sum(gaussian **2)
        # gaussian *= factor  


        title = f'{name.replace("_", " ").title()}'
        if n_layers:
            title += f' ({n_layers} Layers)'
        ax[i].plot(centers, pdf, 'r--', label=f'Gaussian Fit\nμ = {mean_pdf:.4f}, σ = {std_pdf:.4f}')
        ax[i].set_title(title)
        ax[i].set_xlabel(name.replace("_", " ").title())
        ax[i].set_ylabel('Density')
        ax[i].legend()
        ax[i].grid()

    if n_layers:
        plt.suptitle(f"Shape Metrics Comparison for {n_layers} Layers", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()



def plot_hist_with_gaussian_fit(data, label, bins=100, smooth=True):
    data = np.array(data)
    mu, std = norm.fit(data)

    # Histogram
    counts, bin_edges = np.histogram(data, bins=bins, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

    if smooth:
        y_fit = savgol_filter(counts, window_length=11, polyorder=3)
    else:
        y_fit = counts

    # Gaussian curve for comparison
    x_fit = np.linspace(min(data), max(data), 1000)
    gaussian = norm.pdf(x_fit, mu, std)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(bin_centers, y_fit, label='Data (smoothed)', lw=2)
    plt.plot(x_fit, gaussian, 'r--', label=f'Gaussian Fit\nμ = {mu:.4f}, σ = {std:.4f}')
    plt.xlabel(label)
    plt.ylabel('Density')
    plt.title(f'{label} Distribution with Gaussian Fit')
    plt.legend()
    plt.grid(True)
    plt.show()

    return mu, std


# Main analysis
datafile = 'Sample_2476/cluster_density_analysis/{}l_gr.csv'
i = [2, 3, 4, 5]

for number_of_layers in i:
    df = pd.read_csv(datafile.format(number_of_layers))
    
    circularities, name1     = get_shape_metric_info(df['Circularities'])
    feret_diameters, name2   = get_shape_metric_info(df['Feret_Diameter'])
    aspect_ratios, name3     = get_shape_metric_info(df['Aspect_Ratio'])
    clusters_area, name4     = get_shape_metric_info(df['Clusters'])
    mean_circularity = np.array(circularities).mean()
    mean_feret_diameter = np.array(feret_diameters).mean()
    mean_aspect_ratio = np.array(aspect_ratios).mean()
    mean_clusters_area = np.array(clusters_area).mean()
    print(f"Mean Circularity for {number_of_layers} layers: {mean_circularity:.4f}")
    print(f"Mean Feret Diameter for {number_of_layers} layers: {mean_feret_diameter:.4f}")
    print(f"Mean Aspect Ratio for {number_of_layers} layers: {mean_aspect_ratio:.4f}")
    print(f"Mean Clusters Area for {number_of_layers} layers: {mean_clusters_area:.4f}")



    plot_shape_metric_comparison([
        (circularities, name1),
        (feret_diameters, name2),
        (aspect_ratios, name3),
        (clusters_area, name4),
    ], n_layers=number_of_layers, as_line=False)

In [ ]:
%matplotlib qt5
import numpy as np
import cv2
from scipy.optimize import curve_fit
from scipy.ndimage import maximum_filter, gaussian_filter
import matplotlib.pyplot as plt
from imageanalysis.data_loader import DataLoader

# -----------------------------
# Load & crop your image
# -----------------------------
for_debugging_area = '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-23_After_LaserCleaning_PtEvaporation/stack.h5'
img = DataLoader(for_debugging_area)
img = img[0]
atom_img = img[97:97+100, 126:126+100] 
print("Crop shape:", atom_img.shape)

# -----------------------------
# Multi-Gaussian model
# -----------------------------
def multi_2D_Gaussian(coords, *params):
    """
    Sum of n 2D Gaussians with a common offset.
    params = [amp1, xo1, yo1, sigma_x1, sigma_y1, theta1,
              amp2, xo2, yo2, sigma_x2, sigma_y2, theta2,
              ...,
              offset]
    """
    x, y = coords
    g = np.zeros_like(x, dtype=float)
    n = (len(params)-1) // 6  # number of Gaussians
    offset = params[-1]
    
    for i in range(n):
        amp, xo, yo, sigma_x, sigma_y, theta = params[6*i:6*(i+1)]
        xo = float(xo)
        yo = float(yo)
        a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
        b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
        c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
        g += amp * np.exp(-(a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    
    return (g + offset).ravel()

# -----------------------------
# Peak detection for initialization
# -----------------------------
def detect_peaks(img, threshold_rel=0.3, max_peaks=5):
    """
    Detect approximate peak locations in an image.
    """
    smooth = gaussian_filter(img, sigma=1)
    local_max = (smooth == maximum_filter(smooth, size=5))
    candidates = np.argwhere(local_max & (smooth > threshold_rel*smooth.max()))
    
    # Sort by intensity (strongest peaks first)
    intensities = smooth[candidates[:,0], candidates[:,1]]
    sorted_idx = np.argsort(intensities)[::-1]
    candidates = candidates[sorted_idx]
    
    return candidates[:max_peaks]

# -----------------------------
# Fit function
# -----------------------------
def fit_atom_multi(atom_img, ngauss=3):
    nrows, ncols = atom_img.shape
    x = np.arange(0, ncols, 1)
    y = np.arange(0, nrows, 1)
    xv, yv = np.meshgrid(x, y)

    # --- detect peaks for initial guesses ---
    peak_coords = detect_peaks(atom_img, threshold_rel=0.4, max_peaks=ngauss)
    if len(peak_coords) < ngauss:
        ngauss = len(peak_coords)

    guess = []
    bounds_lower, bounds_upper = [], []
    for i, (yo, xo) in enumerate(peak_coords[:ngauss]):
        amp_guess = atom_img.max() - atom_img.min()
        sigma_guess = 2.0
        theta_guess = 0.0
        guess.extend([amp_guess, xo, yo, sigma_guess, sigma_guess, theta_guess])
        
        # Bounds for each Gaussian
        bounds_lower.extend([0, 0, 0, 0.5, 0.5, -np.pi/4])
        bounds_upper.extend([atom_img.max()*2, ncols, nrows, 10, 10, np.pi/4])

    # background offset
    guess.append(atom_img.min())
    bounds_lower.append(0)
    bounds_upper.append(atom_img.max()*2)

    # --- curve fitting ---
    popt, pcov = curve_fit(
        multi_2D_Gaussian, (xv, yv), atom_img.ravel(),
        p0=guess, bounds=(bounds_lower, bounds_upper),
        maxfev=20000
    )
    return popt, (xv, yv), ngauss

# -----------------------------
# Example usage
# -----------------------------
popt, (xv, yv), ngauss = fit_atom_multi(atom_img, ngauss=3)

# Build fitted image
fit_img = multi_2D_Gaussian((xv, yv), *popt).reshape(atom_img.shape)

# Integrated intensities per Gaussian
for i in range(ngauss):
    amp, xo, yo, sigma_x, sigma_y, theta = popt[6*i:6*(i+1)]
    integ = 2*np.pi * amp * sigma_x * sigma_y
    print(f"Gaussian {i+1}: Integrated intensity = {integ:.2f}")

# Plot
fig, axs = plt.subplots(1, 3, figsize=(12,4))
axs[0].imshow(atom_img, cmap='inferno'); axs[0].set_title("Cropped atom")
axs[1].imshow(fit_img, cmap='inferno'); axs[1].set_title("Multi-Gaussian fit")
axs[2].imshow(atom_img-fit_img, cmap='bwr'); axs[2].set_title("Residual")
plt.show()


In [ ]:
import numpy as np
import cv2
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

calibration = 0.007829  # nm/pixel

# --- Gaussian fitting model ---
def twoD_Gaussian(coords, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = coords
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp(-(a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) + c*((y-yo)**2)))
    return g.ravel()

def fit_atom(atom_img):
    nrows, ncols = atom_img.shape
    x = np.arange(0, ncols, 1)
    y = np.arange(0, nrows, 1)
    x, y = np.meshgrid(x, y)
    initial_guess = (atom_img.max()-atom_img.min(), ncols/2, nrows/2, 2, 2, 0, atom_img.min())
    popt, _ = curve_fit(twoD_Gaussian, (x, y), atom_img.ravel(), p0=initial_guess)
    return popt


params = fit_atom(atom_img)
amp, xo, yo, sigma_x, sigma_y, theta, offset = params

# --- Pixel-based area (simple threshold) ---
_, mask = cv2.threshold(atom_img.astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
pixel_area = np.count_nonzero(mask) * (calibration**2)

# --- Gaussian effective areas ---
A_sigma = np.pi * sigma_x * sigma_y * (calibration**2)
A_FWHM = np.pi * (1.178*sigma_x) * (1.178*sigma_y) * (calibration**2)

print(f"Pixel area (nm²): {pixel_area:.4f}")
print(f"Gaussian 1σ ellipse area (nm²): {A_sigma:.4f}")
print(f"Gaussian FWHM ellipse area (nm²): {A_FWHM:.4f}")


In [ ]:
# STEM single-atom / cluster quantification helpers
# -------------------------------------------------
# This cell defines READY-TO-USE functions to extract:
# 1) Integrated HAADF cross-section (threshold-free)
# 2) FWHM-based and α-isophote areas from Gaussian PSF fits
# 3) Cluster fitting (sum of multiple Gaussians with shared sigma)
#
# Usage (quick start, once you have a cropped ROI "roi" around one atom):
#   params = fit_single_gaussian(roi)
#   out = summarize_fit(params, calib_nm_per_px=0.007829, alpha=0.5)
#   print(out)
#
# For clusters:
#   centers = find_peaks_2d(roi, min_distance=3, threshold_abs=roi.mean()+2*roi.std())
#   params_cluster = fit_cluster_shared_sigma(roi, centers)
#   report = summarize_cluster(roi, params_cluster, calib_nm_per_px=0.007829, alpha=0.5)
#   print(report)

import numpy as np
import cv2
from scipy.optimize import curve_fit
from scipy.ndimage import maximum_filter, gaussian_filter

# -----------------------------
# 2D Gaussian building blocks
# -----------------------------
def gaussian2d_rotated(xy, amp, x0, y0, sx, sy, theta, offset):
    """
    Single rotated 2D Gaussian + constant offset.
    xy: (X, Y) meshgrid
    amp: peak above offset (>=0)
    (x0, y0): center (in pixels)
    sx, sy: standard deviations (pixels, >0)
    theta: rotation (radians)
    offset: constant background
    Returns raveled image.
    """
    X, Y = xy
    ct, st = np.cos(theta), np.sin(theta)
    a = (ct**2)/(2*sx**2) + (st**2)/(2*sy**2)
    b = -st*ct/(2*sx**2) + st*ct/(2*sy**2)
    c = (st**2)/(2*sx**2) + (ct**2)/(2*sy**2)
    return (offset + amp * np.exp(-(a*(X-x0)**2 + 2*b*(X-x0)*(Y-y0) + c*(Y-y0)**2))).ravel()

def multi_gaussian2d_shared_sigma(xy, params, shared_theta=False):
    """
    Sum of K Gaussians with shared (sx, sy) and shared offset.
    params structure:
        [sx, sy, offset,  amp1, x1, y1,  amp2, x2, y2,  ...]
    If shared_theta=True, also passes a shared theta at the start:
        [sx, sy, theta, offset,  amp1, x1, y1, ...]
    """
    X, Y = xy
    idx = 0
    sx = params[idx]; idx+=1
    sy = params[idx]; idx+=1
    if shared_theta:
        theta = params[idx]; idx+=1
    else:
        theta = 0.0
    offset = params[idx]; idx+=1

    img = np.zeros_like(X, dtype=float)
    ct, st = np.cos(theta), np.sin(theta)
    a = (ct**2)/(2*sx**2) + (st**2)/(2*sy**2)
    b = -st*ct/(2*sx**2) + st*ct/(2*sy**2)
    c = (st**2)/(2*sx**2) + (ct**2)/(2*sy**2)
    while idx < len(params):
        amp = params[idx]; x0 = params[idx+1]; y0 = params[idx+2]; idx += 3
        img += amp * np.exp(-(a*(X-x0)**2 + 2*b*(X-x0)*(Y-y0) + c*(Y-y0)**2))
    return (img + offset).ravel()

# -----------------------------
# Single-atom fitting (1 Gaussian)
# -----------------------------
def fit_single_gaussian(atom_img, sigma_init_px=2.0):
    """
    Fit a single rotated 2D Gaussian + offset to a cropped ROI around one atom.
    Returns fitted parameters: (amp, x0, y0, sx, sy, theta, offset)
    """
    img = np.asarray(atom_img, dtype=float)
    nrows, ncols = img.shape
    y = np.arange(nrows); x = np.arange(ncols)
    X, Y = np.meshgrid(x, y)

    # Initial guesses
    amp0 = img.max() - img.min()
    x0 = (ncols-1)/2
    y0 = (nrows-1)/2
    sx0 = sy0 = float(sigma_init_px)
    theta0 = 0.0
    offset0 = img.min()

    p0 = (amp0, x0, y0, sx0, sy0, theta0, offset0)

    # Bounds: enforce positive sigmas and reasonable centers
    lower = (0,         0,         0,       0.3, 0.3, -np.pi/2, img.min()-abs(amp0))
    upper = (np.inf, ncols-1, nrows-1,     10.0,10.0,  np.pi/2, img.max())

    popt, pcov = curve_fit(gaussian2d_rotated, (X, Y), img.ravel(), p0=p0, bounds=(lower, upper), maxfev=20000)
    return popt

def summarize_fit(popt, calib_nm_per_px, alpha=0.5):
    """
    Given fitted single-Gaussian params and calibration (nm/px), compute:
      - Integrated intensity (cross-section proxy): S = 2π * amp * sx * sy
      - FWHM_x, FWHM_y (in nm)
      - FWHM ellipse area (nm^2)
      - alpha-isophote area (nm^2), where alpha in (0,1], default 0.5 (FWHM level for Gaussian)
    """
    amp, x0, y0, sx, sy, theta, offset = popt
    # Integrated intensity (model above background)
    S = 2*np.pi * amp * sx * sy

    # FWHM
    f = 2*np.sqrt(2*np.log(2))
    FWHM_x_nm = f * sx * calib_nm_per_px
    FWHM_y_nm = f * sy * calib_nm_per_px

    # FWHM ellipse area
    A_FWHM_nm2 = (np.pi/4) * FWHM_x_nm * FWHM_y_nm

    # alpha-isophote area: A_alpha(px^2)=2π sx sy ln(1/alpha) -> convert to nm^2
    A_alpha_nm2 = (2*np.pi*sx*sy*np.log(1/alpha)) * (calib_nm_per_px**2)

    return {
        "amp": amp,
        "sigma_x_px": sx,
        "sigma_y_px": sy,
        "theta_rad": theta,
        "offset": offset,
        "Integrated_intensity_S": S,
        "FWHM_x_nm": FWHM_x_nm,
        "FWHM_y_nm": FWHM_y_nm,
        "Area_FWHM_nm2": A_FWHM_nm2,
        f"Area_alpha_{alpha:g}_nm2": A_alpha_nm2,
    }

# -----------------------------
# Peak finding for clusters
# -----------------------------
def find_peaks_2d(img, min_distance=3, threshold_abs=None, footprint_size=3):
    """
    Simple local-maximum finder using a maximum filter.
    Returns a list of (y,x) peak coordinates.
    """
    I = np.asarray(img, dtype=float)
    if threshold_abs is None:
        threshold_abs = I.mean() + 2*I.std()

    # Smooth a bit to reduce noise peaks
    Is = gaussian_filter(I, sigma=1.0)
    # Local maxima
    neighborhood = maximum_filter(Is, size=footprint_size)
    peaks_mask = (Is == neighborhood) & (Is > threshold_abs)

    ys, xs = np.nonzero(peaks_mask)

    # Enforce min distance by greedy selection on intensity
    vals = Is[ys, xs]
    order = np.argsort(-vals)
    selected = []
    for idx in order:
        y0, x0 = ys[idx], xs[idx]
        if all((y0 - y1)**2 + (x0 - x1)**2 >= min_distance**2 for (y1, x1) in selected):
            selected.append((y0, x0))
    return selected

# -----------------------------
# Cluster fit: sum of Gaussians with shared sigma (PSF) & shared offset
# -----------------------------
def fit_cluster_shared_sigma(roi, centers_yx, sigma_init_px=2.0, shared_theta=False):
    """
    Fit K Gaussians (K = len(centers_yx)) with shared (sx, sy) and offset.
    centers_yx: list of (y,x) initial centers in pixel coords of roi.
    Returns fitted params vector for multi_gaussian2d_shared_sigma.
    """
    I = np.asarray(roi, dtype=float)
    nrows, ncols = I.shape
    y = np.arange(nrows); x = np.arange(ncols)
    X, Y = np.meshgrid(x, y)

    # Initial params
    sx0 = sy0 = float(sigma_init_px)
    offset0 = float(np.percentile(I, 5))
    amps0 = []
    for (yy, xx) in centers_yx:
        amps0 += [max(1e-3, I[int(yy), int(xx)] - offset0), float(xx), float(yy)]

    if shared_theta:
        p0 = [sx0, sy0, 0.0, offset0] + amps0
        lower = [0.3, 0.3, -np.pi/2, np.min(I)-abs(np.ptp(I))] + [0, 0, 0]*(len(centers_yx))
        upper = [10., 10.,  np.pi/2,  np.max(I)] + [np.inf, ncols-1, nrows-1]*(len(centers_yx))
    else:
        p0 = [sx0, sy0, offset0] + amps0
        lower = [0.3, 0.3, np.min(I)-abs(np.ptp(I))] + [0, 0, 0]*(len(centers_yx))
        upper = [10., 10., np.max(I)] + [np.inf, ncols-1, nrows-1]*(len(centers_yx))

    popt, pcov = curve_fit(
        lambda xy, *p: multi_gaussian2d_shared_sigma(xy, p, shared_theta=shared_theta),
        (X, Y), I.ravel(), p0=p0, bounds=(lower, upper), maxfev=40000
    )
    return popt

def summarize_cluster(roi, popt, calib_nm_per_px, alpha=0.5, shared_theta=False):
    """
    From fitted cluster params, compute per-atom and total integrated intensities,
    and a PSF-derived area metric (FWHM and alpha-isophote) using the shared sigma.
    """
    idx = 0
    sx = popt[idx]; idx+=1
    sy = popt[idx]; idx+=1
    if shared_theta:
        theta = popt[idx]; idx+=1
    else:
        theta = 0.0
    offset = popt[idx]; idx+=1

    atoms = []
    while idx < len(popt):
        amp = popt[idx]; x0 = popt[idx+1]; y0 = popt[idx+2]; idx += 3
        S = 2*np.pi * amp * sx * sy               # integrated intensity of this Gaussian
        atoms.append({"amp": amp, "x0_px": x0, "y0_px": y0, "S": S})

    # Shared PSF areas
    f = 2*np.sqrt(2*np.log(2))
    FWHM_x_nm = f * sx * calib_nm_per_px
    FWHM_y_nm = f * sy * calib_nm_per_px
    A_FWHM_nm2 = (np.pi/4) * FWHM_x_nm * FWHM_y_nm
    A_alpha_nm2 = (2*np.pi*sx*sy*np.log(1/alpha)) * (calib_nm_per_px**2)

    total_S = sum(a["S"] for a in atoms)
    report = {
        "shared_sigma_x_px": sx,
        "shared_sigma_y_px": sy,
        "shared_theta_rad": theta,
        "offset": offset,
        "per_atom": atoms,
        "total_integrated_intensity_S": total_S,
        "FWHM_x_nm": FWHM_x_nm,
        "FWHM_y_nm": FWHM_y_nm,
        "Area_FWHM_nm2": A_FWHM_nm2,
        f"Area_alpha_{alpha:g}_nm2": A_alpha_nm2,
    }
    return report

# -----------------------------
# (Optional) Helper to build ROI around a given center with padding
# -----------------------------
def crop_centered(img, center_yx, half_size):
    y0, x0 = center_yx
    y0, x0 = int(round(y0)), int(round(x0))
    y1, y2 = y0-half_size, y0+half_size+1
    x1, x2 = x0-half_size, x0+half_size+1
    y1 = max(0, y1); x1 = max(0, x1)
    y2 = min(img.shape[0], y2); x2 = min(img.shape[1], x2)
    return img[y1:y2, x1:x2]

# -----------------------------
# Minimal demo on synthetic data (so you see outputs now):
# -----------------------------

# Synthetic single atom
nx, ny = 41, 41
y = np.arange(ny); x = np.arange(nx)
X, Y = np.meshgrid(x, y)
img_syn = gaussian2d_rotated((X, Y), amp=200, x0=20, y0=21, sx=2.5, sy=2.0, theta=0.0, offset=10).reshape(ny, nx)
# Fit
p_single = fit_single_gaussian(atom_img)
out_single = summarize_fit(p_single, calib_nm_per_px=0.007829, alpha=0.5)
out_single

# # Synthetic cluster of 3 atoms with shared sigma
# img_clu = (
#     gaussian2d_rotated((X, Y), 180, 10, 20, 2.2, 2.0, 0.0, 0) +
#     gaussian2d_rotated((X, Y), 120, 22, 18, 2.2, 2.0, 0.0, 0) +
#     gaussian2d_rotated((X, Y),  90, 29, 26, 2.2, 2.0, 0.0, 0) + 15
# ).reshape(ny, nx)

# centers = find_peaks_2d(img_clu, min_distance=4, threshold_abs=img_clu.mean()+2*img_clu.std())
# p_cluster = fit_cluster_shared_sigma(img_clu, centers, sigma_init_px=2.0, shared_theta=False)
# rep = summarize_cluster(img_clu, p_cluster, calib_nm_per_px=0.007829, alpha=0.5, shared_theta=False)
# print("Cluster (synthetic) summary:", rep)


In [ ]:
# roi is your 2D numpy array around a single Pt atom (e.g., 25×25 px)
calib = 0.007829  # nm/pixel

popt = fit_single_gaussian(atom_img)                     # fit 1 Gaussian + offset
summary = summarize_fit(popt, calib_nm_per_px=calib, alpha=0.5)

summary
# Keys:
#  - Integrated_intensity_S        (cross-section proxy, threshold-free)
#  - FWHM_x_nm, FWHM_y_nm
#  - Area_FWHM_nm2
#  - Area_alpha_0.5_nm2            (α can be changed)
